In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

## 데이터 불러오기

In [3]:
mydata = pd.read_csv("data_main_weather.csv", encoding = 'cp949')

In [4]:
pd.set_option('mode.chained_assignment',  None)

In [5]:
# 이상치 처리1
filtering_outlier = mydata.query('revenue > 180000000')
filtering_outlier_index = list(filtering_outlier.index)

for i in filtering_outlier_index:
    tmp = mydata.loc[i,'mother_code']
    mydata.loc[i,'revenue'] = mydata[mydata['mother_code'] == tmp].sort_values(by='revenue' ,ascending=False)['revenue'].iloc[2]

In [6]:
# 이상치 처리2
filtering = mydata[mydata['revenue']==50000].index
mydata = mydata.drop(filtering)

In [7]:
mydata.dropna(subset=['revenue'], inplace=True)


In [8]:
mydata['revenue']

0          2099000.0
1          4371000.0
2          3262000.0
3          6955000.0
4          6672000.0
            ...     
38299     10157000.0
38300     50929000.0
38301    104392000.0
38302     13765000.0
38303     46608000.0
Name: revenue, Length: 35379, dtype: float64

In [9]:
mydata['exposure'].isnull().sum()

14976

In [10]:
mydata['exposure']= mydata['exposure'].fillna(method='pad')

In [11]:
mydata['exposure']

0        20.0
1        20.0
2        20.0
3        20.0
4        20.0
         ... 
38299    20.0
38300    20.0
38301    20.0
38302    20.0
38303    20.0
Name: exposure, Length: 35379, dtype: float64

In [556]:
"""for i in range(1, len(mydata)):
    if mydata['exposure'] [i] == 0 :
        mydata['exposure'] [i] = mydata['exposure'][i-1]"""

"for i in range(1, len(mydata)):\n    if mydata['exposure'] [i] == 0 :\n        mydata['exposure'] [i] = mydata['exposure'][i-1]"

## 이상치 제거 후 랜덤포레스트 성능 평가(MAPE)

In [34]:
mydata1 = mydata.dropna()
mydata1 = mydata1.drop(['holiday','Unnamed: 0', 'datetime','name', 'code','time', 'year', 'dayofweek','count','precipitation'], axis=1)
mydata1 = pd.get_dummies(mydata1) 

In [13]:
mydata1

,month,date,hour,minute,holiday_1,exposure,mother_code,price,revenue,repeat,...,category_이미용,category_잡화,category_주방,category_침구,sex_남성,sex_무관,sex_여성,installment_무관,installment_무이자,installment_일시불
0,1,1,6,0,2,20.0,100346,39900.0,2099000.0,1,...,0,0,0,0,1,0,0,1,0,0
1,1,1,6,0,2,20.0,100346,39900.0,4371000.0,1,...,0,0,0,0,0,0,1,1,0,0
2,1,1,6,20,2,20.0,100346,39900.0,3262000.0,2,...,0,0,0,0,1,0,0,1,0,0
3,1,1,6,20,2,20.0,100346,39900.0,6955000.0,2,...,0,0,0,0,0,0,1,1,0,0
4,1,1,6,40,2,20.0,100346,39900.0,6672000.0,3,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38299,12,31,23,40,0,20.0,100448,148000.0,10157000.0,2,...,0,0,1,0,0,1,0,1,0,0
38300,1,1,0,0,2,20.0,100448,178000.0,50929000.0,3,...,0,0,1,0,0,1,0,1,0,0
38301,1,1,0,0,2,20.0,100448,168000.0,104392000.0,3,...,0,0,1,0,0,1,0,1,0,0
38302,1,1,0,0,2,20.0,100448,158000.0,13765000.0,3,...,0,0,1,0,0,1,0,1,0,0


In [14]:
mydata1['precipitation'] = np.where(mydata['precipitation'] > 0 , 1, 0) 

In [35]:
mydata1

,month,date,hour,minute,holiday_1,exposure,mother_code,price,revenue,repeat,...,category_이미용,category_잡화,category_주방,category_침구,sex_남성,sex_무관,sex_여성,installment_무관,installment_무이자,installment_일시불
0,1,1,6,0,2,20.0,100346,39900.0,2099000.0,1,...,0,0,0,0,1,0,0,1,0,0
1,1,1,6,0,2,20.0,100346,39900.0,4371000.0,1,...,0,0,0,0,0,0,1,1,0,0
2,1,1,6,20,2,20.0,100346,39900.0,3262000.0,2,...,0,0,0,0,1,0,0,1,0,0
3,1,1,6,20,2,20.0,100346,39900.0,6955000.0,2,...,0,0,0,0,0,0,1,1,0,0
4,1,1,6,40,2,20.0,100346,39900.0,6672000.0,3,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38299,12,31,23,40,0,20.0,100448,148000.0,10157000.0,2,...,0,0,1,0,0,1,0,1,0,0
38300,1,1,0,0,2,20.0,100448,178000.0,50929000.0,3,...,0,0,1,0,0,1,0,1,0,0
38301,1,1,0,0,2,20.0,100448,168000.0,104392000.0,3,...,0,0,1,0,0,1,0,1,0,0
38302,1,1,0,0,2,20.0,100448,158000.0,13765000.0,3,...,0,0,1,0,0,1,0,1,0,0


In [36]:
list(mydata1.columns.values)

['month',
 'date',
 'hour',
 'minute',
 'holiday_1',
 'exposure',
 'mother_code',
 'price',
 'revenue',
 'repeat',
 'm_temp',
 'l_temp',
 'h_temp',
 'category_가구',
 'category_가전',
 'category_건강기능',
 'category_농수축',
 'category_생활용품',
 'category_속옷',
 'category_의류',
 'category_이미용',
 'category_잡화',
 'category_주방',
 'category_침구',
 'sex_남성',
 'sex_무관',
 'sex_여성',
 'installment_무관',
 'installment_무이자',
 'installment_일시불']

In [37]:
col_list = ['month',
 'date',
 'hour',
 'minute',
 'holiday_1',
 'exposure',
 'mother_code',
 'price',
 'repeat',
 'm_temp',
 'l_temp',
 'h_temp',
 'category_가구',
 'category_가전',
 'category_건강기능',
 'category_농수축',
 'category_생활용품',
 'category_속옷',
 'category_의류',
 'category_이미용',
 'category_잡화',
 'category_주방',
 'category_침구',
 'sex_남성',
 'sex_무관',
 'sex_여성',
 'installment_무관',
 'installment_무이자',
 'installment_일시불',
  'revenue']

In [38]:
mydata1 = mydata1.reindex(columns = col_list)

In [39]:
"""
# 트레이닝 테스트 셋
X = mydata1.iloc[:,:-1]
y = mydata1.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=156)
"""

'\n# 트레이닝 테스트 셋\nX = mydata1.iloc[:,:-1]\ny = mydata1.iloc[:,-1]\n\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=156)\n'

In [40]:
mydata1[mydata1['month']!=6]

,month,date,hour,minute,holiday_1,exposure,mother_code,price,repeat,m_temp,...,category_잡화,category_주방,category_침구,sex_남성,sex_무관,sex_여성,installment_무관,installment_무이자,installment_일시불,revenue
0,1,1,6,0,2,20.0,100346,39900.0,1,-2.1,...,0,0,0,1,0,0,1,0,0,2099000.0
1,1,1,6,0,2,20.0,100346,39900.0,1,-2.1,...,0,0,0,0,0,1,1,0,0,4371000.0
2,1,1,6,20,2,20.0,100346,39900.0,2,-2.1,...,0,0,0,1,0,0,1,0,0,3262000.0
3,1,1,6,20,2,20.0,100346,39900.0,2,-2.1,...,0,0,0,0,0,1,1,0,0,6955000.0
4,1,1,6,40,2,20.0,100346,39900.0,3,-2.1,...,0,0,0,1,0,0,1,0,0,6672000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38299,12,31,23,40,0,20.0,100448,148000.0,2,-3.8,...,0,1,0,0,1,0,1,0,0,10157000.0
38300,1,1,0,0,2,20.0,100448,178000.0,3,-0.9,...,0,1,0,0,1,0,1,0,0,50929000.0
38301,1,1,0,0,2,20.0,100448,168000.0,3,-0.9,...,0,1,0,0,1,0,1,0,0,104392000.0
38302,1,1,0,0,2,20.0,100448,158000.0,3,-0.9,...,0,1,0,0,1,0,1,0,0,13765000.0


In [41]:
X_test = mydata1[mydata1['month']==6].iloc[:,:-1]
X_train = mydata1[mydata1['month']!=6].iloc[:,:-1]
y_test = mydata1[mydata1['month']==6].iloc[:,-1]
y_train = mydata1[mydata1['month']!=6].iloc[:,-1]

In [42]:
y_train

0          2099000.0
1          4371000.0
2          3262000.0
3          6955000.0
4          6672000.0
            ...     
38299     10157000.0
38300     50929000.0
38301    104392000.0
38302     13765000.0
38303     46608000.0
Name: revenue, Length: 32519, dtype: float64

In [43]:
X_test

,month,date,hour,minute,holiday_1,exposure,mother_code,price,repeat,m_temp,...,category_이미용,category_잡화,category_주방,category_침구,sex_남성,sex_무관,sex_여성,installment_무관,installment_무이자,installment_일시불
16109,6,1,1,0,1,20.0,100380,39900.0,1,18.9,...,0,0,0,0,1,0,0,1,0,0
16110,6,1,1,0,1,20.0,100380,39900.0,1,18.9,...,0,0,0,0,0,0,1,1,0,0
16111,6,1,1,20,1,20.0,100380,39900.0,2,18.9,...,0,0,0,0,1,0,0,1,0,0
16112,6,1,1,20,1,20.0,100380,39900.0,2,18.9,...,0,0,0,0,0,0,1,1,0,0
16113,6,1,1,40,1,13.0,100380,39900.0,3,18.9,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19177,6,30,22,20,1,20.0,100098,119000.0,1,23.8,...,0,0,0,0,0,1,0,1,0,0
19178,6,30,22,40,1,20.0,100098,119000.0,2,23.8,...,0,0,0,0,0,1,0,1,0,0
19179,6,30,23,0,1,20.0,100098,119000.0,3,23.8,...,0,0,0,0,0,1,0,1,0,0
19180,6,30,23,20,1,20.0,100664,79900.0,1,23.8,...,0,0,0,0,0,1,0,1,0,0


In [48]:
rf = RandomForestRegressor(random_state=0, n_estimators=500, max_features = 29)

In [49]:
rf.fit(X_train, y_train)

RandomForestRegressor(max_features=29, n_estimators=500, random_state=0)

In [50]:
y_pred = rf.predict(X_test)

In [51]:
y_pred_over = rf.predict(X_train)

In [52]:
y_pred[0:30]

array([ 5617938.        ,  5883244.        , 10173120.        ,
       10748748.        , 10669705.2       , 10395165.5       ,
        9734409.93333333, 18796991.2       , 30375146.53333333,
       17271890.4       , 35510540.        , 42603184.33333334,
       39613478.        , 61882623.        , 75032662.33333334,
       12890862.        , 12600167.66666667, 50497188.        ,
       53117750.        , 14711051.9       , 11822687.63333333,
       31830707.13333333, 27284652.97619047, 46394737.63333333,
       33542108.56666666, 22495318.66666666, 49952690.66666667,
       59178575.        , 16315425.76666667, 29486237.33333334])

In [53]:
np.array(y_test)[0:30]

array([ 4497000.,  2968000., 10164000.,  6025000.,  7010000.,  5620000.,
       11954000., 15705000., 21884000., 11685000., 27955000., 37021000.,
       52972000., 79733000., 96380000., 19602000., 19503000., 55647000.,
       71581000., 17823000., 14832000., 28914000., 24728000., 30201000.,
       25004000.,  7078000., 27023000., 48719000., 19735000., 28965000.])

In [54]:
# MAPE 평가 지표
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [55]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [56]:
mean_absolute_percentage_error(y_test, y_pred)

56.06894021397169

In [57]:
mean_absolute_percentage_error(y_train, y_pred_over)

28.584415755302555

------------------

## 이상치 제거 후 랜덤포레스트 성능 평가(RMSE)

In [161]:
y_target = mydata1['revenue']
y_target.fillna(0, inplace=True)
X_data = mydata1.drop(['revenue'], axis=1)
#X_data = data.drop(['revenue', 'Unnamed: 0', 'datetime','name', 'code','time','h_temp','l_temp'], axis=1, inplace=False)
#X_data = data.drop(['revenue', 'Unnamed: 0', 'datetime','name', 'code','time','h_temp','l_temp','precipitation'], axis=1, inplace=False)
X_data.fillna(0, inplace=True)

In [162]:
X_data = pd.get_dummies(X_data)

In [163]:
rf = RandomForestRegressor(random_state=0, n_estimators=100)

In [164]:
neg_mse_scores = cross_val_score(rf, X_data, y_target, scoring="neg_mean_squared_error", cv=20)
rmse_scores = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

KeyboardInterrupt: 

In [ ]:
print('10교차 검증의 개별 Negative MSE scores: ', np.round(neg_mse_scores, 2))
print('10교차 검증의 개별 RMSE scores: ', np.round(rmse_scores, 2))
print('10교차 검증의 평균 RMSE scores: {0:.3f}'.format(avg_rmse))